## Load the necessary modules

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import os
import numpy as np
import pandas as pd
import shap
import pickle
import ipywidgets as widgets
from IPython.display import display, HTML
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from collections import namedtuple
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.kernel_ridge import KernelRidge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from skrebate import MultiSURFstar
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import mean_squared_error, r2_score
from matbench.data_ops import mean_absolute_percentage_error
shap.initjs()

In [ ]:
#Load the pandas dataframe with targets and features for training model
df = pd.read_pickle('./dataforml_automatminer.pkl')

## The code block below changes the inputs to model training based on user selection

**<font color='red'>Important note</font>:** 
The user first needs to select either one of the button below to reproduce the results of the associated model presented in paper. 

In [ ]:
# Code block to select model training input data 
heading = widgets.HTML('<h3>Please select the model you want to train and evaluate before procceding</h3>')
include_button = widgets.Button(description='Including LOBSTER features', 
                                layout=widgets.Layout(width='300px', height='50px'),
                               style={'font_weight': 'bold','font_size': '16px', 'border_radius': '300px'})


exclude_button = widgets.Button(description='Excluding LOBSTER features',
                               layout=widgets.Layout(width='300px', height='50px'),
                               style={'font_weight': 'bold', 'font_size': '16px','border_radius': '300px'})

parent = os.getcwd() # get the directory of script


# Define the button click event handlers
def include_button_clicked(b):
    '''
    This callback function includes the ICOHP features for model training and changes the output directory
    to store the results
    '''
    global y, X
    os.chdir(parent)
    isExist = os.path.exists('inc_icohp')
    if not isExist:
        os.mkdir('inc_icohp')
        os.chdir('inc_icohp')
    else:
        os.chdir('inc_icohp')
        
    y=df.iloc[:,0] #targets
    X=df.iloc[:,1:] #features
    print("LOBSTER features will be included in the model evaluation, results will be stored in the 'inc_icohp' directory")
    print("Great! Now you have the necessary data for model training and evaluation. Run the consequent code blocks")
    print("")
def exclude_button_clicked(b):
    '''
    This callback function excludes the ICOHP features for model training and changes the output directory
    to store the results
    '''
    global y, X
    os.chdir(parent)
    isExist = os.path.exists('exc_icohp')
    if not isExist:
        os.mkdir('exc_icohp')
        os.chdir('exc_icohp')
    else:
        os.chdir('exc_icohp')
    
    y=df.iloc[:,0] #targets
    X=df.iloc[:,1:-18] #features
    print("LOBSTER features will be excluded in the model evaluation, results will be stored in the 'exc_icohp' directory")
    print("Great! Now you have the necessary data for model training and evaluation. Run the consequent code blocks")
    print("")
# Attach the event handlers
include_button.on_click(include_button_clicked)
exclude_button.on_click(exclude_button_clicked)

# Display the widgets
display(heading)
display(include_button)
display(exclude_button)
display(HTML('<div class="alert-warning"><h3>Please ensure you selected either of options presented above. You will encounter error ahead as inputs necessary for model will not be instantiated if you don\'t select either of the options</h3></div>'))

In [ ]:
# List with model name that will be trained and evaluated
models_names =['RandomForestRegressor']

# Dict with model hypterparameter
models_dicts = {
     RandomForestRegressor(n_jobs=24): {
         'selector__n_features_to_select': [50, 100, 180], # Number of features to select
        'regressor__n_estimators': [500],
        }
    }

In [ ]:
def get_feature_importance_plot(gridsearchcv_obj, modelname, features, iteration):
    '''
    Function to save the plot of selected features by feature selection algorithm (MultiSurfstar) with its scores
    '''

    selected_feature_indices = np.argsort(-1*gridsearchcv_obj.best_estimator_.steps[0][1].feature_importances_)[:
                                                gridsearchcv_obj.best_estimator_.steps[0][1].n_features_to_select]

    # Get the names of the selected features
    feature_names = [features.columns[i] for i in selected_feature_indices]
    feature_score = [gridsearchcv_obj.best_estimator_.steps[0][1].feature_importances_[i] 
              for i in selected_feature_indices]


    # Create and save plot to neptune logger        
    fig_feat = go.Figure(data=go.Bar(
        x=feature_score,
        y=feature_names,
        orientation='h',
    ))
    fig_feat.update_layout(yaxis = dict(tickfont = dict(size=11)))
    fig_feat.update_layout(xaxis = dict(tickfont = dict(size=11)))
    fig_feat.update_yaxes(title_font=dict(size=22), color='black')
    fig_feat.update_xaxes(title_font=dict(size=22), color='black')
    fig_feat.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True, showgrid=False)
    fig_feat.update_yaxes(showline=True, linewidth=1, linecolor='black', mirror=True, showgrid=False)
    fig_feat.update_xaxes(ticks="inside", tickwidth=1, tickcolor='black', ticklen=5)
    fig_feat.update_yaxes(ticks="inside", tickwidth=1, tickcolor='black', ticklen=5)
    fig_feat.update_layout(template='simple_white')
    fig_feat.update_layout(width=1000, height =1000)
    fig_feat.update_layout(title_text='Feature scores', title_x=0.5)
    fig_feat.write_html("{}/{}_features_{}.html".format(modelname, modelname, iteration),include_mathjax = 'cdn')
    
    return fig_feat

In [ ]:
def get_train_test_plot(errors_test,errors_train,labels, modelname):
    '''
    Function to save the absolute error distribution violin plot
    '''
    
    fig_val = go.Figure()

    fig_val.add_trace(go.Violin(x0=name,
                            y=error_data_test,
                            legendgroup='Test',name='Test',
                            side='positive',
                            hovertext= list(np.concatenate(labels)),
                            line_color='blue', box_visible=True)
                 )
    fig_val.add_trace(go.Violin(x0=name,
                            y=error_data_train,
                            legendgroup='Train',name='Train',
                            side='negative',
                            line_color='orange', box_visible=True)
                 )
    fig_val.update_traces(meanline_visible=True)
    fig_val.update_layout(violingap=0, violinmode='overlay')
    fig_val.update_traces(marker_opacity=0.75)
    fig_val.update_layout(yaxis = dict(tickfont = dict(size=11)))
    fig_val.update_layout(xaxis = dict(tickfont = dict(size=11)))
    fig_val.update_yaxes(title_font=dict(size=22), color='black')
    fig_val.update_xaxes(title_font=dict(size=22), color='black')
    fig_val.update_layout(width=1000, height =1000)
    fig_val.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True, showgrid=False)
    fig_val.update_yaxes(showline=True, linewidth=1, linecolor='black', mirror=True, showgrid=False)
    fig_val.update_xaxes(ticks="inside", tickwidth=1, tickcolor='black', ticklen=5)
    fig_val.update_yaxes(ticks="inside", tickwidth=1, tickcolor='black', ticklen=5)
    fig_val.update_layout(yaxis = dict(tickfont = dict(size=18)))
    fig_val.update_layout(yaxis_title="Validation Absolute error")
    fig_val.update_layout(yaxis = dict(tickfont = dict(size=18)))
    fig_val.update_layout(xaxis = dict(tickfont = dict(size=18)))
    fig_val.update_layout(template='simple_white')
    fig_val.update_layout(yaxis_zeroline=False)
    fig_val.write_html("{}/{}_validation.html".format(modelname,modelname),include_mathjax = 'cdn')

    
    return fig_val

In [ ]:
def get_shap_plot(model, X_train, iteration, modelname):
    '''
    Function to extract and store the shapley values plot to identify feature influence on model predictions 
    '''
    
    selected_feature_indices = np.argsort(-1*model.best_estimator_.steps[0][1].feature_importances_)[:
                                model.best_estimator_.steps[0][1].n_features_to_select]

    # Get the names of the selected features
    feature_names = [X_train.columns[i] for i in selected_feature_indices]
    feature_score = [model.best_estimator_.steps[0][1].feature_importances_[i] 
              for i in selected_feature_indices]

    #Extract shapley values from the best model
    
    explainer = shap.TreeExplainer(model.best_estimator_.steps[1][1], X_train.filter(feature_names))
    shap_values = explainer.shap_values(X_train.filter(feature_names), check_additivity=False)
    
    fig = shap.summary_plot(shap_values, features=X_train.filter(feature_names), feature_names=X_train.filter(feature_names).columns, 
                            show=False)
    plt.savefig('{}/{}_{}.svg'.format(modelname,modelname,iteration))
    plt.close()
    return None

In [ ]:
def grid_search(model, param, X_train, y_train):
    '''
    Convenience function to setup inner cv pipeline for hyperparamter tuning 
    '''
    
    # configure the cross-validation procedure
    cv_inner = KFold(n_splits=5, shuffle=True, random_state=18012019)
    
    #setup model training pipeline
    pipeline = Pipeline([
        ('selector', MultiSURFstar(n_jobs=-1)),
        ('regressor', model)
        ])
    # define search
    search = GridSearchCV(pipeline, param_grid=param, scoring='neg_mean_absolute_error', cv=cv_inner, refit=True, return_train_score=True, n_jobs=24)
    # execute search
    result = search.fit(X_train, y_train)
    # get the best performing model fit on the whole training set
    return result.best_estimator_, search

In [ ]:
def get_metrics_df(abs_errors, rmse_scores, r2_scores, mape_scores, model):
    '''
    Convenience function that evaluates cross validated model performance statistics and returns a pandas dataframe 
    '''
    #define the dataframe
    df = pd.DataFrame(index=[model])
    
    # map of metrics
    metrics = {
        'mae': abs_errors,
        'max_error': abs_errors,
        'rmse': rmse_scores,
        'r2': r2_scores,
        'mape': mape_scores
    }
    
    
    def compute_stats(scores, metric_name):
        '''
        Wrapper function to evaluate statistics using numpy methods 
        '''
        
        if metric_name=='mae':
            return {
                'mean': np.mean(np.mean(scores, axis=1)),
                'max': np.max(np.mean(scores, axis=1)),
                'min': np.min(np.mean(scores, axis=1)),
                'std': np.std(np.mean(scores, axis=1)),
            }
        
        if metric_name=='max_error':
            return {
                'mean': np.mean(np.max(scores, axis=1)),
                'max': np.max(scores),
                'min': np.min(scores),
                'std': np.std(np.max(scores, axis=1)),
            }
        
        else:
            return {
                'mean': np.mean(scores),
                'max': np.max(scores),
                'min': np.min(scores),
                'std': np.std(scores)
            }

    #loop to calculate and populate the metrics in the dataframe
    for metric, scores in metrics.items():
        for subset in ['train', 'test']:
            stats = compute_stats(scores=getattr(scores, subset),metric_name=metric)
            for stat_name, value in stats.items():
                df.loc[name, '{}_{}_{}'.format(metric,subset,stat_name)] = value    
    return df

In [ ]:
# define convinience named tuples to store raw metric data of nested cv runs
abs_errors = namedtuple("abs_errors", "train test")
rmse_scores = namedtuple("rmse_scores", "train test")
r2_scores = namedtuple("r2_scores", "train test")
mape_scores = namedtuple("mape_scores", "train test")

trained_models={}
for name, (model, param) in zip(models_names, models_dicts.items()):
    isExist = os.path.exists(name)
    if not isExist:
        os.mkdir(name)
    print(name + ' model training')

    abs_errors = namedtuple("abs_errors", "train test")
    rmse_scores = namedtuple("rmse_scores", "train test")
    r2_scores = namedtuple("r2_scores", "train test")
    mape_scores = namedtuple("mape_scores", "train test")

    cv_outer = KFold(n_splits=5, shuffle=True, random_state=18012019)

    #store outfold metrics
    test_rmse = []
    train_rmse =[]
    test_r2 = []
    train_r2 =[]
    test_errors=[]
    test_labels=[]
    train_errors=[]
    mape_train=[]
    mape_test=[]
    iteration=1
    # enumerate splits
    for train_ix, test_ix in cv_outer.split(X):
        # split data
        X_train, X_test = X.iloc[train_ix, :].values, X.iloc[test_ix, :].values
        y_train, y_test = y.iloc[train_ix].values, y.iloc[test_ix].values
        
        #get the best model and gridsearch cv object
        best_model, search = grid_search(model, param, X_train, y_train)
        
        # get train set predictions of best model
        y_hat_train  = best_model.predict(X_train)
        
        # evaluate model on the hold out test dataset
        y_hat_test = best_model.predict(X_test)
        
        # evaluate the model performace metrics on train and test sets
        rmse_test = mean_squared_error(y_test, y_hat_test, squared=False)
        rmse_train = mean_squared_error(y_train, y_hat_train, squared=False)

        r2_test = r2_score(y_test, y_hat_test)
        r2_train = r2_score(y_train, y_hat_train)

        test_error = abs(y_test - y_hat_test)
        train_error = abs(y_train - y_hat_train)

        # store the result of each folds
        test_rmse.append(rmse_test)
        train_rmse.append(rmse_train)

        test_r2.append(r2_test)
        train_r2.append(r2_train)

        test_errors.append(test_error)
        test_labels.append(y.iloc[test_ix].index)
        train_errors.append(train_error)

        mape_train.append(mean_absolute_percentage_error(y_pred=y_hat_train, y_true=y_train))
        mape_test.append(mean_absolute_percentage_error(y_pred=y_hat_test, y_true=y_test))

        
        # pickle the trained models  
        filename = '{}/{}_bestmodel_{}.pkl'.format(name, name, iteration)
        pickle.dump(best_model, open(filename, 'wb'))

        #save shapley values plot for each fold
        get_shap_plot(modelname=name,X_train=X.iloc[train_ix, :],model=search, iteration=iteration)

        #save feature importance plot with scores from feature selection algorithm
        get_feature_importance_plot(gridsearchcv_obj=search, modelname=name, features=X.iloc[train_ix, :], 
                                               iteration=iteration)

        
        print('>acc={}, est={}, cfg={}'.format(rmse_test, search.best_score_, search.best_params_))
        print('Mean RMSE: {} (std : {})'.format(np.mean(test_rmse), np.std(test_rmse)))


        iteration+=1

    error_data_test = np.concatenate(test_errors)
    error_data_train = np.concatenate(train_errors)

    # store test and train absolute errors as violin+box plot
    get_train_test_plot(errors_test=error_data_test, errors_train=error_data_train, 
                                  labels=test_labels,modelname=name)

    # populate the defined named tuples with raw data of model performace
    train_test_errors = abs_errors(train_errors, test_errors)
    train_test_rmse = rmse_scores(train_rmse, test_rmse)
    train_test_r2 = r2_scores(train_r2, test_r2)
    train_test_mape = mape_scores(mape_train, mape_test)

    # pass the named tuple to obtain summarized model performace metrics pandas dataframe
    stats_df = get_metrics_df(abs_errors=train_test_errors, rmse_scores=train_test_rmse,
                       r2_scores=train_test_r2, mape_scores=train_test_mape, model=name)

    # save the summary stats as csv
    stats_df.to_csv('summary_results.csv')
    os.chdir('..')
    
    print(name + ' done')
    print('')